# E7 - Project
Team member -
Rutuparn
Tanish
Vihang
Samarpit

# Reading images from the folder


In [2]:
import cv2
import os
import numpy as np

def load_and_process_images(folder_path='images', target_size=(800, 600)):
    """
    Loads all images from a folder.
    - Crops to 4:3 aspect ratio if needed.
    - Scales down to 800x600 if larger, but never scales up.
    - Draws 8x8 grid lines.
    - Saves to 'resized_images'.
    - Divides into 8x8 tiles (for further processing).
    """
    supported_extensions = ('.jpg', '.jpeg', '.png', '.bmp', '.tiff')
    images = []
    resized_images = []
    divided_images = []

    # Create output folder
    output_folder = 'resized_images'
    os.makedirs(output_folder, exist_ok=True)

    if not os.path.exists(folder_path):
        raise FileNotFoundError(f"The folder '{folder_path}' does not exist.")

    for filename in os.listdir(folder_path):
        if filename.lower().endswith(supported_extensions):
            file_path = os.path.join(folder_path, filename)
            try:
                img = cv2.imread(file_path)
                if img is None:
                    print(f"⚠️ Skipped unreadable image: {filename}")
                    continue

                h, w = img.shape[:2]
                aspect_ratio = w / h

                # --- a. Enforce 4:3 aspect ratio by cropping ---
                target_ratio = 4 / 3
                if abs(aspect_ratio - target_ratio) > 0.01:  # allow slight tolerance
                    if aspect_ratio > target_ratio:
                        # Image is too wide → crop width
                        new_w = int(h * target_ratio)
                        x_start = (w - new_w) // 2
                        img = img[:, x_start:x_start + new_w]
                    else:
                        # Image is too tall → crop height
                        new_h = int(w / target_ratio)
                        y_start = (h - new_h) // 2
                        img = img[y_start:y_start + new_h, :]

                # --- b. Scale down if larger than 800x600 ---
                h, w = img.shape[:2]
                if w > target_size[0] or h > target_size[1]:
                    img = cv2.resize(img, target_size, interpolation=cv2.INTER_AREA)

                # --- c. Do NOT scale up smaller images ---
                final_img = img
                h, w = final_img.shape[:2]

                # Draw 8x8 grid lines
                tile_h, tile_w = h // 8, w // 8
                for i in range(1, 8):
                    # Horizontal lines
                    y = i * tile_h
                    cv2.line(final_img, (0, y), (w, y), (0, 255, 0), 1)
                    # Vertical lines
                    x = i * tile_w
                    cv2.line(final_img, (x, 0), (x, h), (0, 255, 0), 1)

                # Save processed image
                save_path = os.path.join(output_folder, filename)
                cv2.imwrite(save_path, final_img)

                # Keep copies in memory
                images.append(final_img.copy())
                resized_images.append(final_img)

                # Divide into 8x8 tiles (using actual dimensions)
                tiles = [
                    final_img[i * tile_h:(i + 1) * tile_h, j * tile_w:(j + 1) * tile_w]
                    for i in range(8)
                    for j in range(8)
                ]
                divided_images.append(tiles)

            except Exception as e:
                print(f"❌ Error processing {filename}: {e}")

    return images, resized_images, divided_images


# ----------------------------
# Example usage
# ----------------------------
if __name__ == "__main__":
    images, resized_images, divided_images = load_and_process_images('images')

    print(f"✅ Loaded and processed {len(images)} images.")
    print(f"📁 Grid-marked images saved in the 'resized_images' folder.")
    if divided_images:
        print(f"Each image divided into {len(divided_images[0])} tiles (8x8).")


✅ Loaded and processed 465 images.
📁 Grid-marked images saved in the 'resized_images' folder.
Each image divided into 64 tiles (8x8).


# Conversion into 800x600 pixels


# dividing into 8x8 grid

Converting the images to greycode to reduce dimentionality

# Image processing to identify animals using HOG.

In [ ]:
import cv2
import os
import numpy as np

def safe_imread(file_path):
    """
    Safely read image using OpenCV with fallback modes.
    """
    try:
        img = cv2.imread(file_path)
        if img is None:
            return cv2.imread(file_path, cv2.IMREAD_REDUCED_COLOR_2)
        return img
    except Exception:
        return cv2.imread(file_path, cv2.IMREAD_REDUCED_COLOR_4)


def crop_to_aspect_ratio(img, target_aspect=(4, 3)):
    """
    Crop the image to a target aspect ratio (4:3 by default).
    """
    h, w = img.shape[:2]
    target_ratio = target_aspect[0] / target_aspect[1]
    current_ratio = w / h

    if abs(current_ratio - target_ratio) < 1e-2:
        return img  # Already 4:3

    if current_ratio > target_ratio:
        # Too wide — crop horizontally
        new_w = int(h * target_ratio)
        start_x = (w - new_w) // 2
        cropped = img[:, start_x:start_x + new_w]
    else:
        # Too tall — crop vertically
        new_h = int(w / target_ratio)
        start_y = (h - new_h) // 2
        cropped = img[start_y:start_y + new_h, :]

    return cropped


def resize_image(img, max_size=(800, 600)):
    """
    Resize the image to a maximum of 800x600.
    Do not scale up smaller images.
    """
    h, w = img.shape[:2]
    max_w, max_h = max_size
    if w > max_w or h > max_h:
        return cv2.resize(img, (max_w, max_h), interpolation=cv2.INTER_AREA)
    return img


def load_and_save_grayscale_images(input_folder='images', output_folder='grey_images'):
    """
    Load all images from input_folder, crop to 4:3, resize down if needed,
    convert to grayscale, and save into output_folder.
    """
    supported_extensions = ('.jpg', '.jpeg', '.png', '.bmp', '.tiff')
    os.makedirs(output_folder, exist_ok=True)

    for filename in os.listdir(input_folder):
        if not filename.lower().endswith(supported_extensions):
            continue

        file_path = os.path.join(input_folder, filename)
        img = safe_imread(file_path)
        if img is None:
            print(f"⚠️ Skipped unreadable image: {filename}")
            continue

        # Step 1: Crop to 4:3
        img = crop_to_aspect_ratio(img, target_aspect=(4, 3))

        # Step 2: Resize only if larger than 800x600
        img = resize_image(img, max_size=(800, 600))

        # Step 3: Convert to grayscale
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

        # Step 4: Save grayscale image
        save_path = os.path.join(output_folder, filename)
        cv2.imwrite(save_path, gray)

        print(f"✅ Saved grayscale image: {filename}")

    print(f"\n🎯 All grayscale images saved in '{output_folder}/'.")


# Example usage
if __name__ == "__main__":
    load_and_save_grayscale_images('images', 'grey_images')


In [1]:
import cv2
import os
import numpy as np

def draw_grid_on_gray_image(img, grid_size=(8, 8)):
    """
    Draw an 8x8 grid with small numbers in the bottom-left corner of each cell.
    Works for grayscale images.
    """
    h, w = img.shape[:2]
    grid_h, grid_w = grid_size
    dy, dx = h // grid_h, w // grid_w

    # Convert grayscale to BGR to draw colored text/lines
    img_colored = cv2.cvtColor(img, cv2.COLOR_GRAY2BGR)

    cell_num = 1
    for i in range(grid_h):
        for j in range(grid_w):
            x, y = j * dx, i * dy
            # Draw green grid rectangles
            cv2.rectangle(img_colored, (x, y), (x + dx, y + dy), (0, 255, 0), 1)
            # Draw small red number on bottom-left of each cell
            cv2.putText(img_colored, str(cell_num),
                        (x + 4, y + dy - 4),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.35, (0, 0, 255), 1, cv2.LINE_AA)
            cell_num += 1

    return img_colored


def process_grey_images_with_grid(input_folder='grey_images', output_folder='grey_images_with_grid'):
    """
    Loads grayscale images, draws 8x8 grid with numbering, and saves them to output_folder.
    """
    supported_extensions = ('.jpg', '.jpeg', '.png', '.bmp', '.tiff')
    os.makedirs(output_folder, exist_ok=True)

    for filename in os.listdir(input_folder):
        if not filename.lower().endswith(supported_extensions):
            continue

        file_path = os.path.join(input_folder, filename)
        img = cv2.imread(file_path, cv2.IMREAD_GRAYSCALE)

        if img is None:
            print(f"⚠️ Skipped unreadable image: {filename}")
            continue

        # Draw grid and numbering
        grid_img = draw_grid_on_gray_image(img, grid_size=(8, 8))

        # Save to output folder
        save_path = os.path.join(output_folder, filename)
        cv2.imwrite(save_path, grid_img)
        print(f"✅ Saved grid image: {filename}")

    print(f"\n🎯 All images with grid saved to '{output_folder}/'.")


# Example usage
if __name__ == "__main__":
    process_grey_images_with_grid('grey_images', 'grey_images_with_grid')


✅ Saved grid image: 100820081692.jpg
✅ Saved grid image: 100820081693.jpg
✅ Saved grid image: 100820081694.jpg
✅ Saved grid image: 100820081695.jpg
✅ Saved grid image: 141120142185.jpg
✅ Saved grid image: 26102010062.jpg
✅ Saved grid image: 29032008693.jpg
✅ Saved grid image: 29032008695.jpg
✅ Saved grid image: 29032008696.jpg
✅ Saved grid image: 29032008701.jpg
✅ Saved grid image: CIMG0001~2.JPG
✅ Saved grid image: CIMG0002.JPG
✅ Saved grid image: CIMG0005.JPG
✅ Saved grid image: CIMG0006.JPG
✅ Saved grid image: CIMG0011.JPG
✅ Saved grid image: CIMG0012~2.JPG
✅ Saved grid image: CIMG0014(1).JPG
✅ Saved grid image: CIMG0014.JPG
✅ Saved grid image: CIMG0015.JPG
✅ Saved grid image: CIMG0017.JPG
✅ Saved grid image: CIMG0018.JPG
✅ Saved grid image: CIMG0021~2.JPG
✅ Saved grid image: CIMG0029.JPG
✅ Saved grid image: CIMG0030.JPG
✅ Saved grid image: CIMG0034.JPG
✅ Saved grid image: CIMG0035.JPG
✅ Saved grid image: CIMG0047.JPG
✅ Saved grid image: CIMG0054.JPG
✅ Saved grid image: CIMG0056.JPG